# **Importing required libraries**




In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
import cv2

import tensorflow as tf
import  tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.models import Model
from tensorflow.keras.layers import Dropout
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, SeparableConv2D
import  tensorflow.keras.layers as layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix, classification_report

ModuleNotFoundError: No module named 'cv2'

# Loading Dataset


In [ ]:
import pickle
pickle_in = open("D:/C dac_DBDA/MY_project/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)



## Splitting Dataset

In [ ]:
x_train,x_1,y_train,y_1=train_test_split(X,y,test_size=0.2,random_state=10)
x_val,x_test,y_val,y_test=train_test_split(x_1,y_1,test_size=0.5,random_state=10)
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

#Model Building

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Dropout


from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, SeparableConv2D

def vgg_depthwise_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = SeparableConv2D(filters = n_filters, kernel_size = (3,3), padding='same', activation='relu')(layer_in)
    #layer_in=Dropout(0.2)(layer_in)
	# add max pooling layer
	layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in
 
visible = Input(shape=(300, 300, 3))
layer = vgg_depthwise_block(visible, 64, 2)
layer = vgg_depthwise_block(layer, 128, 1)
#layer = vgg_depthwise_block(layer, 128, 2)
#layer = vgg_depthwise_block(layer, 256, 2)
layer=Dropout(0.2)(layer)
layer = Flatten()(layer)
layer = Dense(units=4, activation="softmax")(layer)
# create model
model = Model(inputs=visible, outputs=layer)
 
# summarize model
model.summary()
 
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")
 


## Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        fill_mode='constant', cval = 0.0)

datagen.fit(x_train)

history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=32),
                                  epochs = 128, 
                                  validation_data = (x_val,y_val),
                                  verbose = 1)


## Building final model with best validation accuracy

In [ ]:
datagen1 = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        fill_mode='constant', cval = 0.0)

datagen1.fit(x_train)

history_final1 = model.fit_generator(datagen1.flow(x_train,y_train, batch_size=32),
                                  epochs = 50, 
                                  validation_data = (x_val,y_val),
                                  verbose = 1)



## Training vs validation graphs

In [ ]:
plt.plot(history_final1.history['acc'], label="train_accuracy")
plt.plot(history_final1.history['val_acc'], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig('/content/seprable1.jpg')
plt.show()

plt.plot(history_final1.history['loss'], label="train_loss")
plt.plot(history_final1.history['val_loss'], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig('/content/seprable2.jpg')
plt.show()


## Test Accuracy

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)


In [ ]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


## Confusion Matrix

In [ ]:
pred = model.predict(x_test)
df1=pd.DataFrame(pred)

In [ ]:
pred_final=df1.idxmax(axis=1)

pred_final

In [ ]:
cm = confusion_matrix(y_test, pred_final)

In [ ]:
plt.figure(dpi=70)
sns.heatmap(cm, annot=True, cmap="Blues")
plt.xticks(ticks=[0.5,1.5,2.5,3.5], labels=["Chickenpox", "Measles", "Monkeypox","Normal"])
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=["Chickenpox", "Measles", "Monkeypox","Normal"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig('/content/confusion_matrix.jpg')
plt.show()




#Saving Model

In [ ]:
model.save("/content/drive/MyDrive/model1.h5")
print("Saved model to disk")